# Risk calculator process

## Importing modules for running process
In this section all modules which are used by the system are loaded in memory

In [1]:
# Import libraries
import os
import glob
import numpy as np
import pandas as pd

# Import modules
import deforestation as defo

## Setting global parameters
In this section all configurations about folders and files to set up
the system are created. These configurations are global variables for the system

In [2]:
# Global folders
#root_folder = "/home/jovyan/work/BID/dci_fauno/"
root_folder = "D:\\CIAT\\Code\\BID\\dci_fauno"
data_folder = os.path.join(root_folder, "data","risk_calculator")
inputs = os.path.join(data_folder,"inputs")
outputs = os.path.join(data_folder,"outputs")
conf_folder = os.path.join(data_folder,"conf")
# Deforestation folders
def_folder = os.path.join(inputs,"def")

# Parameters file
conf_xls = pd.ExcelFile(os.path.join(conf_folder,"conf.xlsx"))
conf = conf_xls.parse("conf")

# Creating folders
# Create folder for deforestation files
if not os.path.exists(def_folder):    
    os.mkdir(def_folder)

print("Done!")

Done!


# Deforestation
This chapter is responsible to prepare all files about deforestation in order to they can 
be processed then.

## Download and unzip deforestation data
In this section all data about deforestation for a country will be downloaded and extracted
into specific folders. It keeps all files

In [ ]:
# Getting parameters for downloading deforestation files
url_base = conf.loc[conf["parameter"] == "def_src_base_url","value"].values[0]
file_name_base = conf.loc[conf["parameter"] == "def_src_file_name","value"].values[0]
def_periods = conf.loc[conf["parameter"] == "def_periods","value"].values[0].split(",")

# Download files from source in zip format and extract them
defo.download_deforestation(def_folder, url_base, file_name_base, def_periods)

## Extract deforestation values
This section takes the raster, which were downloaded, so extract just deforestation pixels
and create a new raster with this data. Then it creates a summary raster which merge all
others raster files in just one

In [ ]:
# Getting parameters for filtering deforestation pixels
def_value = int(conf.loc[conf["parameter"] == "def_value","value"].values[0])
def_pixel_size = float(conf.loc[conf["parameter"] == "def_pixel_size","value"].values[0])

defo.extract_deforestation(def_folder, def_value, def_pixel_size)
defo.summary_deforestation(def_folder)
print("Done!")

Calculating parameters for new files
Outputs paramaters for rasters files:
Bounds: POLYGON ((-66.65601560981048 -4.244791035761322, -66.65601560981048 13.41161398742386, -81.79485170083258 13.41161398742386, -81.79485170083258 -4.244791035761322, -66.65601560981048 -4.244791035761322))
CSR: EPSG:4686


d:\ciat\virtualenv\jupyter\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2010_2012\Geotiff\DCCB_SMBYC_CBBQ_V5_2010_2012.tif
Cropping raster
Resampling raster
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2010_2012.tif
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2012_2013\Geotiff\DCCB_SMBYC_CBBQ_V5_2012_2013.tif
Cropping raster
Resampling raster
Saving: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\fixed\DCCB_SMBYC_CBBQ_V5_2012_2013.tif
Opening: D:\CIAT\Code\BID\dci_fauno\data\risk_calculator\inputs\def\content\Cambio_Cobertura_Bosque_No_Bosque_2013_2014\Geotiff\DCCB_SMBYC_CBBQ_V6_2013_2014.tif
Cropping raster


In [ ]:
import rasterio as rio
from matplotlib import pyplot
import numpy as np
from rasterio.merge import merge
%matplotlib inline


        
    
    
"""    
print("Merging files")
mosaic, out_trans = merge(raster_list)
    
meta_ref.update({"driver": "GTiff",
                        "height": mosaic.shape[1],
                        "width": mosaic.shape[2],
                        "transform": out_trans,
                        "crs": meta_ref.crs})
dest_file = os.path.join(outputs_folder, "summary.tif")
print("Saving:" + dest_file)
with rio.open(dest_file, "w", **meta_ref) as dest:
    dest.write(mosaic)
"""

## Codigo andres

In [ ]:
import rasterio as rio
from shapely.geometry import Polygon
from rasterio.coords import BoundingBox

def getraster_boundingbox(metadata):
    # get boundaries
    img_bounds = list(rio.transform.array_bounds(metadata['height'], transform=metadata['transform'],width=metadata['width']))
    ## organice data
    img_bounds = BoundingBox(left=img_bounds[0], bottom=img_bounds[1], right=img_bounds[2], top=img_bounds[3])

    image_wkt = [(img_bounds[0], img_bounds[3]),
                 (img_bounds[0], img_bounds[1]),
                 (img_bounds[2], img_bounds[1]),
                 (img_bounds[2], img_bounds[3])]

    return image_wkt
def re_sample(raster_data, raster_ref_path=None, sampling_type="nearest", metadata=None, metadaref=None):
    # check if the input is a path string or if is an array
    if type(raster_data) == str:
        raster_data = rasterio.open(raster_data)
        metadata = raster_data.profile.copy()

        affine_val = raster_data.profile.copy()['transform']
        out_kwargs = metadata
        npdata = raster_data.read(1)
        countlayers = raster_data.count

    elif metadata is not None:
        npdata = raster_data
        out_kwargs = metadata
        affine_val = metadata['affine']
        if len(npdata.shape) == 3:
            countlayers = npdata.shape[0]
        else:
            countlayers = 1
    else:
        raise ValueError('Invalid raster input');

    ## get imgage bounds

    img_bounds = gis_functions.getraster_boundingbox(out_kwargs)

    ## get metadata reference
    if raster_ref_path is not None:
        raster_ref = rasterio.open(raster_ref_path)
        meta_ref = raster_ref.profile.copy()
    elif metadaref is not None:
        meta_ref = metadaref
    else:
        raise ValueError('Invalid raster reference');

    ## copy destination parameters
    dst_driver = meta_ref['driver']
    dst_crs = meta_ref['crs']
    dst_transform = meta_ref['transform']

    dst_height = meta_ref['height']
    dst_width = meta_ref['width']


    ## condition to validate their geometry reference
    if dst_crs != out_kwargs['crs']:
        out_kwargs, npdata = project_raster(npdata, out_kwargs, dst_crs)

        affine_val = out_kwargs['transform']


    ## create an empty numpy array
    newarr = np.empty(shape=(countlayers, dst_height, dst_width))

    ## update profile metada with new data
    out_kwargs.update({
        'driver': dst_driver,
        'crs': dst_crs,
        'transform': dst_transform,
        #'affine': dst_affine,
        'width': dst_width,
        'height': dst_height,
        'count': countlayers
    })
    meta_ref.update({
        'dtype': 'uint16'
    })
    ## change raster type
    srcnodata = out_kwargs['nodata']

    if (npdata.dtype != meta_ref['dtype']):
        if out_kwargs['dtype'] == 'uint16' or out_kwargs['dtype'] == "uint8" :
            meta_ref.update({'dtype': 'uint16'})
            out_kwargs.update({
                'dtype': 'uint16'
            })
            npdata = npdata.astype(np.uint16)
            newarr = newarr.astype(np.uint16)
            srcnodata = 0

        if out_kwargs['dtype'] == 'int16':
            out_kwargs.update({
                'nodata': 0,
                'dtype': 'uint16'
            })
            newarr = newarr.astype(np.uint16)

        if (out_kwargs['dtype'] == 'float64') or (out_kwargs['dtype'] == 'float32'):

            meta_ref.update({'dtype': 'float32'})
            npdata = npdata.astype(np.float32)
            newarr = newarr.astype(rasterio.float32)
            srcnodata = out_kwargs['nodata']
            out_kwargs.update({
                'nodata': -9999,
                'dtype': 'float32'
            })
            npdata = np.round(npdata, decimals=4)

    elif npdata.dtype == "uint16" or npdata.dtype == "uint8":
        newarr = newarr.astype(np.uint16)
        srcnodata = 0
        out_kwargs.update({
            'nodata': 0,
            'dtype': 'uint16'
        })


    if 'blockxsize' in out_kwargs and dst_width < out_kwargs['blockxsize']:
        del out_kwargs['blockxsize']
    if 'blockysize' in out_kwargs and dst_height < out_kwargs['blockysize']:
        del out_kwargs['blockysize']


    if sampling_type == 'nearest':
        res_method = Resampling.nearest;
    elif sampling_type == 'linear':
        res_method = Resampling.bilinear;
    elif sampling_type == 'cubic':
        res_method = Resampling.cubic;
    else:
        raise ValueError('Invalid resample method');


    reproject(
        source=npdata,
        destination=newarr,
        src_transform=affine_val,
        src_crs=dst_crs,
        src_nodata=srcnodata,
        dst_transform=out_kwargs['transform'],
        dst_crs=out_kwargs['crs'],
        dst_nodata=out_kwargs['nodata'],
        resampling=res_method)

    if raster_ref_path is not None:
        raster_ref.close()

    # if newarr.dtype != npdata.dtype:
    #     if npdata.dtype == "uint16":
    #         newarr.astype(np.uint16)
    #     else:
    #         newarr.astype(np.float32)
    return [out_kwargs, newarr]

def getraster_boundingbox(metadata):
    # get boundaries
    img_bounds = list(
        rasterio.transform.array_bounds(metadata['height'], transform=metadata['transform'],
                                        width=metadata['width']))
    ## organice data
    img_bounds = BoundingBox(left=img_bounds[0], bottom=img_bounds[1], right=img_bounds[2],
                             top=img_bounds[3])
    return img_bounds